In [28]:
import pandas as pd
import datetime
import requests
import json
import math
import numpy as np

## key 저장

In [ ]:
f = open("keyfile.txt", 'r')
key = {}
for i in range(1, 11):
    key[i] = f.readline().rstrip()
    print('key'+str(i))
f.close()

In [ ]:
key

In [ ]:
key[1]

## 데이터 목록
### 1. 영화 목록
* INPUT
    - 키 값, 현재 페이지, 결과 ROW 의 개수, 영화명, 감독명, 조회시작 개봉연도(YYYY), 조회종료 개봉연도(YYYY), 조회시작 제작연도(YYYY), 조회종료 제작연도(YYYY), 국적코드, 영화유형코드

* OUTPUT
    - 영화코드, 영화명(국문),영화명(영문), 제작연도, 개봉연도, 영화유형, 제작상태, 제작국가(전체), 영화장르(전체), 대표 제작국가명, 대표 장르명, 영화감독, 영화감독명, 제작사, 제작사 코드, 제작사명

### 2. 영화 상세정보
* INPUT
    - 키 값, 영화코드
    
* OUTPUT
    - 영화코드, 영화명(국문), 영화명(영문), 영화명(원문), 제작연도, 상영시간, 개봉연도, 제작상태명, 영화유형명, 제작국가, 제작국가명, 장르명, 감독, 감독명, 감독명(영문), 배우, 배우명, 배우명(영문), 배역명, 배역명(영문), 상영형태 구분1, 상영형태 구분2, 상영형태명, 심의정보, 심의번호, 관람등급 명칭, 참여 영화사, 참여 영화사 코드, 참여 영화사명, 참여 영화사명(영문), 참여 영화사 분야명, 스텝, 스텝명, 스텝명(영문), 스텝역할명
    
### 3. 영화사 목록
* INPUT
    - 키 값, 현재 페이지(default : “1”), 결과 ROW 의 개수(default : “10”), 영화사명, 대표자명, 분류코드(default: 전체)
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 영화사 분류, 대표자명, 필모리스트

### 4. 영화사 상세정보
* INPUT
    - 키 값, 영화사코드
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 대표자명, 영화사 분류, 영화사 분류명, 영화사 필모, 참여 영화코드, 참여 영화명, 영화사 참여 분류명

### 5. 영화인 목록
* INPUT
    - 키 값, 현재 페이지를 지정(default : “1”), 결과 ROW 의 개수(default : “10”), 영화인명, 필모리스트
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 분야, 필모리스트

### 6. 영화인 상세정보
* INPUT
    - 키 값, 영화인코드
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 성별, 영화인 분류명, 영화인 필모, 참여 영화코드, 참여 영화명, 참여 분야, 관련 URL


## 1. 영화 리스트들 중에서 2019 박스오피스로 필터링 된 영화들 정보 가져오기(xlsx)

상영 횟수 40회 이하 아웃라이어 제거 위함(영진위 한국영화성평등소위원회 분석 참고). 


In [59]:
movies_2019_filtered = pd.read_excel('KOBIS_boxoffice_2019_filtered.xlsx')
movies_2019_filtered = movies_2019_filtered[:-1]
movies_2019_filtered

,순위,영화명,개봉일,매출액,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
0,252,시간의 종말,2016-11-03,734000,0.000,26471400,292,4102,1,46,한국,한국,인디라인,(주)영화사 진진,전체관람가,다큐멘터리,김대현,NaN
1,189,다방의 푸른 꿈,2017-01-26,1470000,0.000,6351000,625,1480,2,48,한국,한국,인디라인,"스토리 너머,아이 엠",전체관람가,다큐멘터리,김대현,김정구
2,162,스타박'스 다방,2018-01-11,1696000,0.000,15217000,843,2772,1,49,한국,한국,영화사 온난전선,(주)드림팩트엔터테인먼트,15세이상관람가,"코미디,드라마",이상우,"백성현,이상아,서신애,이정구,신원호,서호철,이다슬,이산인,이은욱,이찬양,이철진,장기..."
3,418,영주,2018-11-22,1033500,0.000,192729750,147,24823,4,51,한국,한국,NaN,CGV아트하우스,12세이상관람가,드라마,차성덕,"김향기,김호정,유재명,탕준상,강병걸,강병걸,김기용,김기용,김서영,차진영"
4,204,로드킬,2019-02-21,4239100,0.000,4239100,542,542,17,51,한국,한국,㈜리필름,(주)갤럭시컴퍼니,청소년관람불가,"공포(호러),스릴러",이수성,"이철민,오광록"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,5,나쁜 녀석들: 더 무비,2019-09-11,39520100504,0.050,39520100504,4567533,4567533,1511,113681,한국,한국,"씨제이이앤엠(주),(주)영화사비단길",씨제이이앤엠(주),15세이상관람가,"범죄,액션",손용호,"마동석,김상중,김아중,장기용,강예원,김대영,박형수,김혜윤"
155,6,돈,2019-03-20,28869614062,0.037,28869614062,3389035,3389035,1431,120808,한국,한국,"(주)사나이픽처스,(주)영화사 월광",(주)쇼박스,15세이상관람가,범죄,박누리,"류준열,유지태,조우진,원진아,박명신,이화룡,오희준,정기섭,차래형"
156,2,기생충,2019-05-30,85864656245,0.109,85864656245,10083002,10083002,1948,192829,한국,한국,(주)바른손이앤에이,씨제이이앤엠(주),15세이상관람가,드라마,봉준호,"송강호,이선균,조여정,최우식,박소담,장혜진,이정은,박명훈,현승민,정현준,윤혜리,박근록"
157,3,엑시트,2019-07-31,79150817862,0.100,79150817862,9413862,9413862,1660,202101,한국,한국,"(주)외유내강,(주)필름케이",씨제이이앤엠(주),12세이상관람가,"코미디,액션",이상근,"조정석,윤아,고두심,박인환,김지영,강기영,오희준,배유람,유수빈,조성희,주보비"


In [41]:
movies_2019_filtered.isnull().sum()

순위         0
영화명        0
개봉일        0
매출액        0
매출액        0
누적매출액      0
관객수        0
누적관객수      0
스크린수       0
상영횟수       0
대표국적       0
국적         0
제작사       12
배급사        0
등급         0
장르         0
감독         1
배우         4
dtype: int64

In [78]:
movies_2019_filtered.fillna('-', inplace=True)

In [80]:
movies_2019_filtered[movies_2019_filtered['감독 '] == '-']

,순위,영화명,개봉일,매출액,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
96,35,러브 유어셀프 인 서울,2019-01-26,3209682900,0.004,3209682900,342366,342366,224,4535,한국,한국,"빅히트 엔터테인먼트,씨제이 씨지브이(CJ CGV)(주)",씨제이 씨지브이(CJ CGV)(주),전체관람가,공연,-,"김남준,김석진,민윤기,정호석,박지민,김태형,전정국"


### 영화 기본정보 가져오는 함수

In [81]:
# 페이지 수, 조회시작 개봉연도, 조회종료 개봉연도를 파라미터(입력값)로 지정해준다.
def get_movies(movieNm, year, directorNm):
    # 영화 목록 API JSON URL
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json'
    
    # 입력받은 영화이름, 년도, 감독이름으로 API 호출.
    if directorNm == '-':
        payload = {
            'key': key[1]
            , 'itemPerPage': '100' # 한페이지에 한번에 불러올 수 있는 최대 데이터의 갯수
            , 'movieNm':movieNm
            , 'openStartDt': year
        }
    else:
         payload = {
            'key': key[1]
            , 'itemPerPage': '100' # 한페이지에 한번에 불러올 수 있는 최대 데이터의 갯수
            , 'movieNm':movieNm
            , 'directorNm':directorNm
        }

    # 결과를 변수에 담고 json 형식으로 변환해준다.
    res = requests.get(url, params=payload).json()
    
    # 쓸데 없는 겉부분 버리고 영화 리스트 컨텐츠만 가져옴
    return res['movieListResult']['movieList']

In [92]:
# 함수 테스트 감독명 없을 경우
res = get_movies('극한직업', 2019, '-')

In [93]:
pd.DataFrame(res)

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companys
0,20182530,극한직업,Extreme Job,2018,20190123,장편,개봉,한국,코미디,한국,코미디,[{'peopleNm': '이병헌'}],"[{'companyCd': '20140742', 'companyNm': '(주)어바..."


### 영화 기본정보 가져오는 함수 돌려서 영화 코드 가져오기

In [83]:
# 영화 몇개나 있는지 확인
len(movies_2019_filtered)

159

In [84]:
# 컬럼 이름 틀리지 않도록 확인. *'감독 '* 주의
movies_2019_filtered.columns

Index(['순위', '영화명', '개봉일', '매출액 ', '매출액', '누적매출액 ', '관객수 ', '누적관객수 ', '스크린수 ',
       '상영횟수 ', '대표국적 ', '국적 ', '제작사 ', '배급사 ', '등급 ', '장르 ', '감독 ', '배우 '],
      dtype='object')

In [85]:
# 함수에 필요한 정보(영화명, 감독명, 개봉일) 가져오기
movieNm_list = movies_2019_filtered['영화명']
movieNm_list

0            시간의 종말
1          다방의 푸른 꿈
2          스타박'스 다방
3                영주
4               로드킬
           ...     
154    나쁜 녀석들: 더 무비
155               돈
156             기생충
157             엑시트
158            극한직업
Name: 영화명, Length: 159, dtype: object

In [97]:
# 감독명은 한명만 있어도 검색 가능
directorNm_list = movies_2019_filtered['감독 '].apply(lambda x: x.split(',')[0])
directorNm_list

0      김대현
1      김대현
2      이상우
3      차성덕
4      이수성
      ... 
154    손용호
155    박누리
156    봉준호
157    이상근
158    이병헌
Name: 감독 , Length: 159, dtype: object

In [88]:
# 개봉일은 년도로만 검색 가능
openYear_list = movies_2019_filtered['개봉일'].apply(lambda x: x.year)
openYear_list

0      2016
1      2017
2      2018
3      2018
4      2019
       ... 
154    2019
155    2019
156    2019
157    2019
158    2019
Name: 개봉일, Length: 159, dtype: int64

In [98]:
movie_info_list = []

for movie in zip(movieNm_list, openYear_list, directorNm_list):
    print(movie)
    movie_info_list.append(get_movies(movie[0], movie[1], movie[2])[0])

('시간의 종말', 2016, '김대현')
('다방의 푸른 꿈', 2017, '김대현')
("스타박'스 다방", 2018, '이상우')
('영주', 2018, '차성덕')
('로드킬', 2019, '이수성')
('늙은 자전거', 2015, '문희융')
('할머니의 먼 집', 2016, '이소현')
('동경', 2019, '윤웅대')
('걸작', 2019, '최진욱')
('방문객', 2019, '박영철')
('델타 보이즈', 2017, '고봉수')
('뿔을 가진 소년', 2019, '김휘근')
('나는 원래 대전에서 로맨틱 코미디를 찍으려고 했었다', 2019, '배기원')
('인어전설', 2018, '오멸')
('곤지암', 2018, '정범식')
('콩쥐별전 - 전주성의 비밀', 2019, '배효상')
('폴란드로 간 아이들', 2018, '추상미')
('다영씨', 2018, '고봉수')
('선종 무문관', 2018, '윤용진')
('검은 여름', 2019, '이원영')
('창간호', 2019, '백승환')
('도어락', 2018, '이권')
('부산행', 2016, '연상호')
('포항', 2019, '모현신')
('두 번째 겨울', 2018, '김의곤')
('산상수훈', 2017, '유영의')
('사범', 2019, '조동희')
('공작', 2018, '윤종빈')
('소은이의 무릎', 2019, '최헌규')
('갈까부다', 2019, '고봉수')
('앨리스 죽이기', 2019, '김상규')
('귀신의 향기', 2019, '이준학')
('불빛 아래서', 2019, '조이예환')
('계절과 계절 사이', 2019, '김준식')
('불한당: 나쁜 놈들의 세상', 2017, '변성현')
('타인은 지옥이다', 2019, '이영용')
('안녕, 나의 소녀 시절이여', 2018, '김한석')
('버블 패밀리', 2018, '마민지')
('공포체험단 곤지암편', 2019, '정범식')
('발광하는 현대사', 2019, '조성규')
('국경의 왕', 2019, '임정환'

In [106]:
movie_baisc_info = pd.DataFrame(movie_info_list)
movie_baisc_info

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companys
0,20167821,시간의 종말,For the End of Time,2016,20161103,장편,개봉,한국,다큐멘터리,한국,다큐멘터리,[{'peopleNm': '김대현'}],"[{'companyCd': '20061889', 'companyNm': '인디라인'}]"
1,20153303,다방의 푸른 꿈,Try to Remember,2015,20170126,장편,개봉,한국,다큐멘터리,한국,다큐멘터리,[{'peopleNm': '김대현'}],"[{'companyCd': '20061889', 'companyNm': '인디라인'}]"
2,20161724,스타박'스 다방,Bittersweet Brew,2016,20180111,장편,개봉,한국,"코미디,드라마",한국,코미디,[{'peopleNm': '이상우'}],"[{'companyCd': '20040209', 'companyNm': '영화사 온..."
3,20177341,영주,Youngju,2018,20181122,장편,개봉,한국,드라마,한국,드라마,[{'peopleNm': '차성덕'}],[]
4,20196370,로드킬,ROAD KILL,2019,20190221,장편,개봉,한국,"공포(호러),스릴러",한국,공포(호러),[{'peopleNm': '이수성'}],"[{'companyCd': '20063187', 'companyNm': '㈜리필름'}]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,20188750,나쁜 녀석들: 더 무비,The Bad Guys: Reign of Chaos,2019,20190911,장편,개봉,한국,"범죄,액션",한국,범죄,[{'peopleNm': '손용호'}],"[{'companyCd': '20110854', 'companyNm': '씨제이이앤..."
155,20176103,돈,Money,2018,20190320,장편,개봉,한국,범죄,한국,범죄,[{'peopleNm': '박누리'}],"[{'companyCd': '20122737', 'companyNm': '(주)사나..."
156,20183782,기생충,PARASITE,2019,20190530,장편,개봉,한국,드라마,한국,드라마,[{'peopleNm': '봉준호'}],"[{'companyCd': '20100558', 'companyNm': '(주)바른..."
157,20184621,엑시트,EXIT,2019,20190731,장편,개봉,한국,"코미디,액션",한국,코미디,[{'peopleNm': '이상근'}],"[{'companyCd': '20100374', 'companyNm': '(주)외유..."


<br>\
<br>
### 영화 상세정보 함수

영화코드를 가져왔으니 이제 영화 상세정보를 가져올 수 있습니다.

In [103]:
def get_movie_details(movieCd):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'

    payload = {
        'key': key[1]
        , 'movieCd': movieCd
    }

    res = requests.get(url, params=payload).json()
    
    return res['movieInfoResult']['movieInfo']

영화 기본 정보에서 알 수 있는 정보는 제외, 이외에 상세정보에서만 얻을 수 있는 데이터들 확인.

1. 러닝타임
2. 심의 정보
3. 감독 정보
4. 배우 정보
5. 제작사 정보
6. 스태프 정보

In [105]:
# get_movie_details 함수 테스트
res = get_movie_details(20182530)
res

{'movieCd': '20182530',
 'movieNm': '극한직업',
 'movieNmEn': 'Extreme Job',
 'movieNmOg': '',
 'showTm': '111',
 'prdtYear': '2018',
 'openDt': '20190123',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '한국'}],
 'genres': [{'genreNm': '코미디'}],
 'directors': [{'peopleNm': '이병헌', 'peopleNmEn': 'LEE Byoung-heon'}],
 'actors': [{'peopleNm': '류승룡',
   'peopleNmEn': 'RYU Seung-ryong',
   'cast': '고반장',
   'castEn': ''},
  {'peopleNm': '이하늬', 'peopleNmEn': 'LEE Ha-nee', 'cast': '장형사', 'castEn': ''},
  {'peopleNm': '진선규',
   'peopleNmEn': 'JIN Seon-kyu',
   'cast': '마형사',
   'castEn': ''},
  {'peopleNm': '이동휘',
   'peopleNmEn': 'LEE Dong-hwi',
   'cast': '영호',
   'castEn': ''},
  {'peopleNm': '공명', 'peopleNmEn': 'GONG Myung', 'cast': '재훈', 'castEn': ''},
  {'peopleNm': '양현민',
   'peopleNmEn': 'YANG Hyun-min',
   'cast': '홍상필 ',
   'castEn': ''},
  {'peopleNm': '장진희',
   'peopleNmEn': 'JANG Jin-hee',
   'cast': '선희',
   'castEn': ''},
  {'peopleNm': '허준석',
   'peopleNmEn': 'HEO Ju

### 후가공을 위한 데이터 확인

영화 상세정보 호출 결과 그대로 데이터프레임화 할 수 없으니 필요한 정보들을 종류별로 가져와서 따로따로 저장해서 테이블화 해야겠다.

1. 러닝타임

In [107]:
res['showTm']

'111'

2. 심의 정보

In [108]:
res['audits']

[{'auditNo': '2018-MF02462', 'watchGradeNm': '15세이상관람가'}]

3. 감독 정보
4. 배우 정보
5. 제작사 정보
6. 스태프 정보

In [109]:
res['directors']

[{'peopleNm': '이병헌', 'peopleNmEn': 'LEE Byoung-heon'}]

In [110]:
res['actors']

[{'peopleNm': '류승룡',
  'peopleNmEn': 'RYU Seung-ryong',
  'cast': '고반장',
  'castEn': ''},
 {'peopleNm': '이하늬', 'peopleNmEn': 'LEE Ha-nee', 'cast': '장형사', 'castEn': ''},
 {'peopleNm': '진선규',
  'peopleNmEn': 'JIN Seon-kyu',
  'cast': '마형사',
  'castEn': ''},
 {'peopleNm': '이동휘', 'peopleNmEn': 'LEE Dong-hwi', 'cast': '영호', 'castEn': ''},
 {'peopleNm': '공명', 'peopleNmEn': 'GONG Myung', 'cast': '재훈', 'castEn': ''},
 {'peopleNm': '양현민',
  'peopleNmEn': 'YANG Hyun-min',
  'cast': '홍상필 ',
  'castEn': ''},
 {'peopleNm': '장진희', 'peopleNmEn': 'JANG Jin-hee', 'cast': '선희', 'castEn': ''},
 {'peopleNm': '허준석',
  'peopleNmEn': 'HEO Jun-seok',
  'cast': '정실장',
  'castEn': ''},
 {'peopleNm': '한준우',
  'peopleNmEn': 'HAN Joon-woo',
  'cast': '마포서 강력반1',
  'castEn': ''},
 {'peopleNm': '김명준',
  'peopleNmEn': 'KIM Myung-jun',
  'cast': '정실장 넥타이직원',
  'castEn': ''},
 {'peopleNm': '나철',
  'peopleNmEn': 'NA Chul',
  'cast': '안산분점 조직원 2',
  'castEn': ''},
 {'peopleNm': '정재광',
  'peopleNmEn': 'JEONG Jae-gwang',
 

In [111]:
res['companys']

[{'companyCd': '20140742',
  'companyNm': '(주)어바웃필름',
  'companyNmEn': 'About Film',
  'companyPartNm': '제작사'},
 {'companyCd': '20156628',
  'companyNm': '영화사 해그림 주식회사',
  'companyNmEn': '',
  'companyPartNm': '제작사'},
 {'companyCd': '20110854',
  'companyNm': '씨제이이앤엠(주)',
  'companyNmEn': 'CJ ENM Corp.',
  'companyPartNm': '제작사'},
 {'companyCd': '20110854',
  'companyNm': '씨제이이앤엠(주)',
  'companyNmEn': 'CJ ENM Corp.',
  'companyPartNm': '배급사'},
 {'companyCd': '20110854',
  'companyNm': '씨제이이앤엠(주)',
  'companyNmEn': 'CJ ENM Corp.',
  'companyPartNm': '제공'},
 {'companyCd': '20100109',
  'companyNm': '씨제이엔터테인먼트',
  'companyNmEn': 'CJ Entertainment',
  'companyPartNm': '해외세일즈사'}]

In [112]:
res['staffs']

[{'peopleNm': '김성환', 'peopleNmEn': 'KIM Seong-hwan', 'staffRoleNm': '제작'},
 {'peopleNm': '김미혜', 'peopleNmEn': 'KIM Mi-hye', 'staffRoleNm': '제작'},
 {'peopleNm': '김태성', 'peopleNmEn': 'KIM Tae-seong', 'staffRoleNm': '작사/작곡/편곡'},
 {'peopleNm': '김태성', 'peopleNmEn': 'KIM Tae-seong', 'staffRoleNm': '음악'},
 {'peopleNm': '모성진', 'peopleNmEn': 'Mo Sung-jin', 'staffRoleNm': '제작'},
 {'peopleNm': '이종석', 'peopleNmEn': 'LEE Jong-seok', 'staffRoleNm': '프로듀서'},
 {'peopleNm': '고대석', 'peopleNmEn': '', 'staffRoleNm': '프로듀서'},
 {'peopleNm': '허재영', 'peopleNmEn': '', 'staffRoleNm': '제작팀'},
 {'peopleNm': '김솔비', 'peopleNmEn': 'KIM Sol-bi', 'staffRoleNm': '연출팀'},
 {'peopleNm': '문충일', 'peopleNmEn': '', 'staffRoleNm': '시나리오(각본)'},
 {'peopleNm': '이덕렬', 'peopleNmEn': '', 'staffRoleNm': '촬영팀'},
 {'peopleNm': '이지애', 'peopleNmEn': '', 'staffRoleNm': '소품팀'},
 {'peopleNm': '강정훈', 'peopleNmEn': 'GANG Jeong-hun', 'staffRoleNm': '소품팀'},
 {'peopleNm': '김미혜', 'peopleNmEn': 'KIM Mi-hye', 'staffRoleNm': '기획'},
 {'peopleNm': '김주

### 후가공

1. 상세정보를 확인하고 싶은 영화리스트에서 영화 코드만 뽑아옵니다.
2. for문 돌면서 get_movie_details로 API 호출.
3. 각각의 영화에 대한 API 호출 결과를 받아서 필요한 정보들을 데이터프레임화 할 수 있는 형태로 리스트에 각각 저장
4. 필요한 경우에 영화 코드로 조인해서 하나의 데이터로 사용할 수 있게 각각의 데이터와 영화코드 함께 딕셔너리 형태로 저장

In [116]:
# 영화 코드만 뽑아오기
movieCd_list = movie_baisc_info['movieCd']

In [117]:
movie_showTm = []
movie_audits = []
movie_directors = []
movie_actors = []
movie_staffs = []
movie_companys = []

# 테스트 할 때는 `movieCd_list.iloc[:1]` 하나만 돌려보세요.

# movieCd_list 길이가 3000이 넘어갈 경우 for문을 쪼개서 돌려야 합니다. 함수 정의된 부분으로 가서 key도 바꿔줘야 해요!.
# ex) movieCd_list.iloc[:3000] 이런 식으로 구간을 나눠서 여러번 함수를 실행시켜야겠죠?
for code in movieCd_list:
    res = get_movie_details(code)
    movieNm = res['movieNm']
    print(code)
    

    showTm_dict = {}
    showTm_dict['movieCd'] = code
    showTm_dict['showTm'] = res['showTm']
    movie_showTm.append(showTm_dict)

    for audit in res['audits']:
        audits_dict = {}
        audits_dict['movieCd'] = code
        audits_dict['auditNo'] = audit['auditNo']
        audits_dict['watchGradeNm'] = audit['watchGradeNm']
        movie_audits.append(audits_dict)
    
    for director in res['directors']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = director['peopleNm']
        person_info['repRoleNm'] = '감독'
        movie_directors.append(person_info)
    for actor in res['actors']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = actor['peopleNm']
        person_info['repRoleNm'] = '배우'
        person_info['cast'] = actor['cast']
        movie_actors.append(person_info)
    for staff in res['staffs']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = staff['peopleNm']
        person_info['staffRoleNm'] = staff['staffRoleNm']
        movie_staffs.append(person_info)
    
    for company in res['companys']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['companyCd'] = company['companyCd']
        person_info['companyNm'] = company['companyNm']
        person_info['companyPartNm'] = company['companyPartNm']
        movie_companys.append(person_info)

20167821
20153303
20161724
20177341
20196370
20142403
20158359
20191591
20199023
20179346
20162972
20178203
20190804
20174008
20180804
20194942
20189281
20183390
20183144
20173930
20195963
20179230
20156564
20148617
20160501
20170272
20181301
20179101
20176026
20198978
20172464
20180361
20173383
20181199
20164844
20182325
20181876
20174244
20194044
20193017
20176006
20183374
20193669
20162140
20193928
20183382
20183132
20174284
20177552
20184642
20174042
20191302
20183343
20173993
20186645
20189803
20186652
20183364
20173991
20189647
20186651
20183345
20199346
20191744
20183727
20183001
20187301
20198829
20191105
20183085
20184961
20181198
20191783
20180902
20189649
20188117
20193013
20192689
20183848
20191845
20181104
20190470
20190725
20198043
20193642
20181096
20191141
20188083
20181168
20184243
20196881
20185350
20194045
20168773
20198912
20190703
20185124
20188393
20166301
20188404
20198597
20190741
20188144
20183785
20181103
20197092
20195791
20182241
20166602
20198982
20182501
2

추려진 데이터들 확인

In [144]:
showTm_df = pd.DataFrame(movie_showTm)
showTm_df.to_csv('showTm_2019_ko.csv', index=False)

In [145]:
audits_df = pd.DataFrame(movie_audits)
audits_df.to_csv('audits_2019_ko.csv', index=False)

In [146]:
directors_df = pd.DataFrame(movie_directors)
directors_df.to_csv('directors_2019_ko.csv', index=False)

In [147]:
actors_df = pd.DataFrame(movie_actors)
actors_df.to_csv('actors_2019_ko.csv', index=False)

In [148]:
staffs_df = pd.DataFrame(movie_staffs)
staffs_df.to_csv('staffs_2019_ko.csv', index=False)

In [149]:
companys_df = pd.DataFrame(movie_companys)
companys_df.to_csv('companys_2019_ko.csv', index=False)

### # 주의 # 한 테스크가 끝날 때마다 csv로 저장하고 다음 작업 시작할 때는 저장된 csv 불러와서 시작해야 합니다.

In [150]:
directors_df = pd.read_csv('directors_2019_ko.csv')

In [151]:
actors_df = pd.read_csv('actors_2019_ko.csv')

In [152]:
staffs_df = pd.read_csv('staffs_2019_ko.csv')

In [153]:
companys_df = pd.read_csv('companys_2019_ko.csv')

<br>\
<br>\
<br>
## 영화인 정보 가져오기

위에서 영화 상세정보 데이터에서 감독, 배우, 스태프 각각의 정보를 가져왔죠. 그 정보를 이용해서 영화인의 코드를 알아낼 겁니다. 번거롭지만 영화인 코드는 영화인 세부정보를 가져오기 위해 꼭 필요한 데이터에요.

In [ ]:
def get_staffs(movieNm, peopleNm):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json'

    # 영화인의 기본정보를 가져오기 위해서는 정보를 알고 싶은 사람의 '이름'과 '참여했던 작품 이름'이 있어야 합니다.
    # 영화 이름만으로 검색하면 영화에 참여했던 모든 스태프를 가져올 수 있지만 동명의 영화가 함께 검색되어 특정 영화의 정보를 가져오기에는 무리가 있습니다.
    # 때문에 특정하게 알고싶은 사람이 이름과 작품의 이름으로 특정 사람을 한정지어 가져오는 것이 낫다고 판단하여 짜여진 함수입니다.
    payload = {
        'key': key[1]
        , 'curPage': 1 # 한사람만 검색되므로 1페이지를 넘어가는 경우가 없을 거라고 가정하고 진행
        , 'itemPerPage': '50' # 한사람만 검색되므로 데이터 갯수가 50개를 넘어가는 경우가 없을 거라고 가정하고 진행
        , 'peopleNm': peopleNm
        , 'filmoNames': movieNm
    }

    res = requests.get(url, params=payload).json()
    
    return res['peopleListResult']

In [ ]:
def get_staffs_code(data):

    staffsCd_list = []
    num = 1
    
    # 이름과 작품으로 영화인 정보에 접근할 것을 알고 있었기 때문에 위에서 감독, 배우, 스태프들의 정보를 담을 때 각각의 사람들이 참여했던 작품의 이름도 함께 저장이 되어있습니다.
    # 때문에 directors_df, actors_df, staffs_df 어느것을 넣더라도 똑같이 작동할 수 있어요.
    
    # get_staffs_code()에 directors_df가 입력되면
    # 아래 반복문에서 이름 컬럼과 작품 컬럼을 각각 하나씩 튜플로 묶어 반복문을 실행합니다.

    for movieNm, peopleNm in zip(data['movieNm'], data['peopleNm']):
        person_info = {}
        
        # 한번 for문이 돌때마다 코드를 검색할 사람의 이름과 작품이 뽑히고, 해당 정보를 get_staffs() 함수에 집어넣으면 해당 영화인의 기본정보를 얻을 수 있습니다.
        staff = get_staffs(movieNm, peopleNm)['peopleList'][0]

        person_info['peopleCd'] = staff['peopleCd']
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = peopleNm
        person_info['repRoleNm'] = staff['repRoleNm']
        person_info['filmoNames'] = staff['filmoNames']

        staffsCd_list.append(person_info)
        
        print(peopleNm, num)
        num += 1

    return staffsCd_list

In [156]:
directors_df

,movieCd,movieNm,peopleNm,repRoleNm
0,20167821,시간의 종말,김대현,감독
1,20153303,다방의 푸른 꿈,김대현,감독
2,20161724,스타박'스 다방,이상우,감독
3,20177341,영주,차성덕,감독
4,20196370,로드킬,이수성,감독
...,...,...,...,...
172,20188750,나쁜 녀석들: 더 무비,손용호,감독
173,20176103,돈,박누리,감독
174,20183782,기생충,봉준호,감독
175,20184621,엑시트,이상근,감독


In [157]:
actors_df

,movieCd,movieNm,peopleNm,repRoleNm,cast
0,20153303,다방의 푸른 꿈,김정구,배우,NaN
1,20161724,스타박'스 다방,백성현,배우,박성두
2,20161724,스타박'스 다방,이상아,배우,성주란(성두 이모)
3,20161724,스타박'스 다방,서신애,배우,최연서
4,20161724,스타박'스 다방,이정구,배우,옥경찰
...,...,...,...,...,...
1528,20182530,극한직업,허준석,배우,정실장
1529,20182530,극한직업,한준우,배우,마포서 강력반1
1530,20182530,극한직업,김명준,배우,정실장 넥타이직원
1531,20182530,극한직업,나철,배우,안산분점 조직원 2


In [158]:
staffs_df

,movieCd,movieNm,peopleNm,staffRoleNm
0,20167821,시간의 종말,김대현,프로듀서
1,20167821,시간의 종말,박선미,제작팀
2,20167821,시간의 종말,홍성수,사운드팀
3,20167821,시간의 종말,정이솔,사운드효과
4,20167821,시간의 종말,정수연,사운드믹싱
...,...,...,...,...
6109,20182530,극한직업,이지애,소품팀
6110,20182530,극한직업,강정훈,소품팀
6111,20182530,극한직업,김미혜,기획
6112,20182530,극한직업,김주연,예고편


In [163]:
directors_df[:3]

,movieCd,movieNm,peopleNm,repRoleNm
0,20167821,시간의 종말,김대현,감독
1,20153303,다방의 푸른 꿈,김대현,감독
2,20161724,스타박'스 다방,이상우,감독


In [165]:
#test
res = get_staffs_code(directors_df[:3])
res

김대현
김대현
이상우


[{'peopleCd': '20167301',
  'movieNm': '시간의 종말',
  'peopleNm': '김대현',
  'repRoleNm': '감독',
  'filmoNames': '시간의 종말|다방의 푸른 꿈|한국번안가요사|살인의 강|내 신발에게'},
 {'peopleCd': '20167301',
  'movieNm': '다방의 푸른 꿈',
  'peopleNm': '김대현',
  'repRoleNm': '감독',
  'filmoNames': '시간의 종말|다방의 푸른 꿈|한국번안가요사|살인의 강|내 신발에게'},
 {'peopleCd': '20111446',
  'movieNm': "스타박'스 다방",
  'peopleNm': '이상우',
  'repRoleNm': '감독',
  'filmoNames': "자매의 공동묘지|몽달|열대야|애정소년 잔혹사|스타박'스 다방|어느 여름날 밤에|더티 로맨스|워킹 스트리트|나는 쓰레기다|친애하는 지도자동지께|스피드|비상구|지옥화|욕정이 활활|바비|아버지는 개다|엄마는 창녀다|트로피컬 마닐라|소설, 영화와 만나다"}]

### TODO. get_staffs_code() 함수를 이용해 2019 directors, actors, staffs의 영화인 코드를 가져오기.

* 각각의 데이터 프레임에서 데이터의 갯수를 보고 데이터 프레임 전체를 한 번에 돌릴지 나누어서 몇 번에 돌릴 수 있을지 판단해서 사용하는 key 조정을 해야 합니다.

* get_staffs_code() 함수의 결과로는 리스트가 반환됩니다.

ex) 한번에 돌릴 수 있을 때:
```python
result = get_staffs_code(directors_df)
```

ex) 한번에 돌릴 수 없을 때:
```python
result1 = get_staffs_code(directors_df.iloc[:3000]) # def get_staffs_code가 정의 되어있는 코드에서 key 값을 새 것으로 바꿈.
result2 = get_staffs_code(directors_df.iloc[3000:6000])

result = result1 + result2
```

Q1) 2019 감독들의 영화인 코드 가져오기

In [167]:
directorCd_list = get_staffs_code(directors_df)

김대현
김대현
이상우
차성덕
이수성
문희융
이소현
윤웅대
최진욱
박영철
고봉수
김휘근
배기원
오멸
정범식
배효상
추상미
고봉수
윤용진
이원영
백승환
하수민
정인희
지일주
이권
연상호
모현신
김의곤
유영의
조동희
윤종빈
최헌규
고봉수
김상규
이준학
조이예환
김준식
변성현
이영용
김한석
마민지
정범식
조성규
임정환
임태규
박철우
임흥순
현진식
여균동
김규민
김소영
최국희
장혜영
정희재
김서윤
강이관
부지영
정대건
김중현
유은정
최현영
백승기
최창환
이강현
왕민철
김선웅
박근영
김진홍
최승호
김한라
임오정
강동완
차인표
전혜림
박영주
박주영
신혜진
복운석
정인봉
이승현
안주영
고명성
한가람
이종은
홍상수
정지훈
박일호
원종식
이상훈
하원준
강상우
정다운
진재운
임준현
정귀호
이민환
장형윤
백재호
손미
류장하
이옥섭
정혁기
신상민
김재환
한승우
우민호
김재희
윤가은
정용주
김지혜
김진원
이호경
송원근
유선동
한상호
김보라
이창근
박호찬
박수진
시즈노 코분
조석현
강지은
김성기
고정욱
강형철
박준수
이춘백
오성윤
김유성
임경택
이정호
최신규
유재운
이민재
김진철
최신규
남대중
장규성
이수진
김현호
윤창섭
홍승완
홍성호
최신규
김진철
이정범
김병우
이성태
김윤석
김주호
용수
강윤성
조철현
이계벽
조민호
김태훈
곽경택
한준희
김주환
육상효
정지우
강효진
김한결
정다원
김홍선
권오광
이종언
장재현
엄유나
이한
이원태
원신연
손용호
박누리
봉준호
이상근
이병헌


Q2) 2019 배우들의 영화인 코드 가져오기

In [168]:
actorCd_list = get_staffs_code(actors_df)

김정구
백성현
이상아
서신애
이정구
신원호
서호철
이다슬
이산인
이은욱
이찬양
이철진
장기원
장동주
박정환
김향기
김호정
유재명
탕준상
강병걸
강병걸
김기용
김기용
김서영
차진영
이철민
오광록
최종원
박민상
박상면
조안
춘자
김형범
박삼순
이소현
장춘옥
홍순영
주종혁
이해선
최진욱
주남정
김명건
이주예
박상준
강재철
하명진
전상진
채연희
최근모
민승준
정호
송경의
박기륭
백승환
이웅빈
신민재
김충길
윤지혜
김소영
김정기
고성완
황성균
조하은
최일순
김예나
권기하
이종철
조하영
전혜빈
문희경
이경준
강래연
윤효진
이경준
이경진
이경호
이동규
이동현
이수연
이순옥
이승민
이승은
이승재
이은미
김정원
이은하
이정빈
김지민
성민정
성민철
이주영
이주영
이준혁
이지영
이혜원
김태준
김한결
김현동
김현민
김현수
김현정
김현지
오승범
김희진
김희진
김희철
오해성
유경
문창현
박상균
박서연
박선영
박영희
박유나
장지원
전용우
전유민
정서원
정성우
정수현
정연주
정영희
박주애
박주영
정재영
박지원
김혜영
강지수
고상근
조은
고수빈
김경미
김나연
김나경
김건휘
김경숙
강민석
강민우
김도연
김동우
김동환
김민정
김민주
김민철
최희열
하성민
하성민
한영
허양자
김선미
김수진
김수현
김은옥
김영호
김예림
김수지
김수정
김성우
김성민
위하준
박지현
오아연
문예원
박성훈
이승욱
유제윤
이주원
정아노마
이송
강동완
김나영
김미나
김새롬
이석준
이온유
이혜경
추상미
추상미
신민재
이호정
강하람
김용진
김정기
김충길
박원진
백승환
유현정
윤지혜
이호정
조준희
조준희
고성완
우상전
고동업
안홍진
조용주
우지현
이건우
정형석
문종원
조달환
이지현
조민재
김재화
서동갑
이영진
백수장
지일주
강찬희
백승환
이병수
부진서
백주환
공효진
김예원
김성오
조복래
이가섭
이천희
김동규
김보경
김유정
김지한
김지현
백민준
이상희
이승창
장효석
한지은
배유리
배명진
공유
김정희
정유미
김율호
마동석
최우식
안소희
김의성
김수안
박민우
박민혁
임종훈
박종관
장문성
박혜진
손지훈
손지훈
연상호
조춘호
지동현
윤미중

KeyboardInterrupt: 

Q3) 2019 스태프들의 영화인 코드 가져오기

In [ ]:
staffCd_list1 = get_staffs_code(staffs_df[:3000])

In [ ]:
staffCd_list2 = get_staffs_code(staffs_df[3000:6000])

In [ ]:
staffCd_list3 = get_staffs_code(staffs_df[6000:])

Q4) 2019 감독, 배우, 스태프들의 영화인 코드를 하나의 데이터 프레임으로 합쳐 csv로 저장해주세요. index=False로 저장해주세요.

해당 데이터프레임을 이용해 영화인 상세 데이터를 가져와야 합니다.

각각 csv로 저장하기

In [169]:
directorCd_df = pd.DataFrame(directorCd_list)
directorCd_df.to_csv('directorCd_2019_list.csv', index=False)

In [ ]:
actor_df = pd.DataFrame(actorCd_list)
actor_df.to_csv('actorCd_2019_list.csv', index=False)

In [ ]:
staffCd1_df = pd.DataFrame(staffCd_list1)
staffCd1_df.to_csv('staffCd_2019_list1.csv', index=False)

In [ ]:
staffCd2_df = pd.DataFrame(staffCd_list2)
staffCd2_df.to_csv('staffCd_2019_list2.csv', index=False)

In [29]:
staffCd3_df = pd.DataFrame(staffCd_list3)
staffCd3_df.to_csv('staffCd_2019_list3.csv', index=False)

,peopleCd,movieNm,peopleNm,repRoleNm,filmoNames
0,10004970,타짜: 원 아이드 잭,김영민,프로듀서,양자물리학|타짜: 원 아이드 잭|사라진 밤|그대 이름은 장미|두 남자|용의자|은밀하...
1,10002458,타짜: 원 아이드 잭,권오광,감독,타짜: 원 아이드 잭|범죄의 여왕|굿바이 싱글|돌연변이|소셜포비아|족구왕|세이프|질...
2,20331712,타짜: 원 아이드 잭,김재홍,촬영팀,타짜: 원 아이드 잭|벌새|증인|돌멩이
3,20146130,타짜: 원 아이드 잭,김보묵,아트디렉터,타짜: 원 아이드 잭|나랏말싸미|1987|불한당: 나쁜 놈들의 세상|싱글라이더|비밀...
4,20113179,타짜: 원 아이드 잭,김유경,프로듀서,타짜: 원 아이드 잭|사라진 밤|타짜-신의 손|아따쿨|블리치
...,...,...,...,...,...
109,20163358,극한직업,이지애,소품팀,기묘한 가족|극한직업|창궐|염력|대장 김창수|남한산성|보안관
110,10000853,극한직업,강정훈,소품팀,기묘한 가족|극한직업|창궐|7년의 밤|염력|대장 김창수|남한산성|섬. 사라진 사람들...
111,10004162,극한직업,김미혜,프로듀서,극한직업|리얼|파파|알포인트|피아노 치는 대통령
112,20227383,극한직업,김주연,예고편,나랏말싸미|기생충|어린 의뢰인|왓칭|사자|악질경찰|항거:유관순 이야기|극한직업|말모...


### # 주의 # 한 테스크가 끝날 때마다 csv로 저장하고 다음 작업 시작할 때는 저장된 csv 불러와서 시작해야 합니다.

In [170]:
directorCd_df = pd.read_csv('directorCd_2019_list.csv')

In [187]:
actorCd_df = pd.read_csv('actorCd_2019_list.csv')

In [ ]:
staffCd1_df = pd.read_csv('staffCd_2019_list1.csv')

In [ ]:
staffCd2_df = pd.read_csv('staffCd_2019_list2.csv')

In [171]:
staffCd3_df = pd.read_csv('staffCd_2019_list3.csv')

<br>\
<br>\
<br>
### 영화인 상세 가져오기

대망의 영화인 상세 데이터입니다.

In [174]:
def get_staff_details(peopleCd):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json'

    # 영화인 코드를 이용해 세부 데이터를 불러옵니다.
    payload = {
        'key': key[1]
        , 'peopleCd': peopleCd
    }

    res = requests.get(url, params=payload).json()
    
    return res['peopleInfoResult']['peopleInfo']

TODO) 아래 테스트는 임시 데이터로 한 테스트 입니다. 위에서 Q4번이 해결 됐으면 해당 csv로 치환해주세요.

In [175]:
# 데이터 확인
directorCd_df

,peopleCd,movieNm,peopleNm,repRoleNm,filmoNames
0,20167301,시간의 종말,김대현,감독,시간의 종말|다방의 푸른 꿈|한국번안가요사|살인의 강|내 신발에게
1,20167301,다방의 푸른 꿈,김대현,감독,시간의 종말|다방의 푸른 꿈|한국번안가요사|살인의 강|내 신발에게
2,20111446,스타박'스 다방,이상우,감독,자매의 공동묘지|몽달|열대야|애정소년 잔혹사|스타박'스 다방|어느 여름날 밤에|더티...
3,10071129,영주,차성덕,감독,영주|비밀은 없다|돼지 같은 여자|물 좀 주소|미쓰 홍당무|사라진 밤|울지 않는다
4,10056145,로드킬,이수성,감독,로드킬|대가리|독고다이|일진 3|일진 2|메모리즈|일진|미스터 좀비(감독판)|휴가|...
...,...,...,...,...,...
172,20188569,나쁜 녀석들: 더 무비,손용호,감독,나쁜 녀석들: 더 무비|살인의뢰|사랑 따윈 필요 없어
173,20151353,돈,박누리,조감독,돈|남자가 사랑할 때|베를린|부당거래|서울이 보이냐|더 게임|김관장 대 김관장 대 김관장
174,10031641,기생충,봉준호,감독,"기생충|옥자|해무(海霧)|설국열차|토니 레인즈의 한국영화 25년|마더|감독들, 김기..."
175,20111926,엑시트,이상근,감독,"엑시트|간만에 나온 종각이|명환이 셀카|베이베를 원하세요?|견제부자|감상과 이해, ..."


In [176]:
# 임시로 테스트 데이터를 하나 불러와 저장했습니다.
# 어떤 데이터들이 어떤 형식으로 뽑혀나오는지 확인하고 상세정보를 어떻게 저장할지 설계합니다.
# 성별만 가져올까 하다가 여배우 남배우의 필모리스트도 비교해보면 좋을 듯 하여 저장하겠습니다.
get_staff_detail_test = get_staff_details(20167301)
get_staff_detail_test

{'peopleCd': '20167301',
 'peopleNm': '김대현',
 'peopleNmEn': 'KIM Dae-hyun',
 'sex': '남자',
 'repRoleNm': '감독',
 'homepages': [],
 'filmos': [{'movieCd': '20167821', 'movieNm': '시간의 종말', 'moviePartNm': '감독'},
  {'movieCd': '20167821', 'movieNm': '시간의 종말', 'moviePartNm': '시나리오(각본)'},
  {'movieCd': '20167821', 'movieNm': '시간의 종말', 'moviePartNm': '프로듀서'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '감독'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '시나리오(각본)'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '프로듀서'},
  {'movieCd': '20153303', 'movieNm': '다방의 푸른 꿈', 'moviePartNm': '기획'},
  {'movieCd': '20128611', 'movieNm': '한국번안가요사', 'moviePartNm': '감독'},
  {'movieCd': '20100981', 'movieNm': '살인의 강', 'moviePartNm': '감독'},
  {'movieCd': '20100981', 'movieNm': '살인의 강', 'moviePartNm': '배우'},
  {'movieCd': '20100981', 'movieNm': '살인의 강', 'moviePartNm': '시나리오(각본)'},
  {'movieCd': '20180215', 'movieNm': '내 신발에게', 'moviePartNm': '감독'}]}

In [177]:
# 영화인 코드
get_staff_detail_test['peopleCd']

'20167301'

In [178]:
# 성별
get_staff_detail_test['sex']

'남자'

In [179]:
# 필모그래피 리스트
for movie in get_staff_detail_test['filmos']:
    print(movie['movieCd'])
    print(movie['movieNm'])
    print(movie['moviePartNm'])

20167821
시간의 종말
감독
20167821
시간의 종말
시나리오(각본)
20167821
시간의 종말
프로듀서
20153303
다방의 푸른 꿈
감독
20153303
다방의 푸른 꿈
시나리오(각본)
20153303
다방의 푸른 꿈
프로듀서
20153303
다방의 푸른 꿈
기획
20128611
한국번안가요사
감독
20100981
살인의 강
감독
20100981
살인의 강
배우
20100981
살인의 강
시나리오(각본)
20180215
내 신발에게
감독


In [180]:
# 2019년만 해도 감독, 배우, 스태프의 총 합은 3000을 훨씬 웃도는 숫자 입니다.
# 횟수 제한 문제로 3000번을 기준으로 잘라서 정보들을 저장한 후 나중에 합쳐주도록 하겠습니다.
# key 사용횟수가 3000회 이하로 남아있다면 가능 횟수에 맞춰서 코드 조정해주세요.
# 중간에 key 사용 가능 횟수가 모두 끝나면 저장하고 다른날 다시 불러와서 이어서 작업 해주세요.

director_detail_list = []
num = 1

for staffCd in directorCd_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    # 성별 리스트와 필모리스트를 따로 만들까 하다가 한 사람의 세부 정보인데 굳이 그럴 필요가 있나 싶어서 한꺼번에 저장합니다.
    # 필모리스트에 있는 영화들을 기준으로 해당 영화인이 참여 했던 작품 하나당 데이터 하나가 생성됩니다.
    # 데이터 한 행에는 '영화인코드, 이름, 성별, 참여영화코드, 참여영화이름, 참여했던역할(감독, 배우, 스태프 등)'이 들어갑니다.
    for movie in get_staff_detail_test['filmos']:
        person_info = {}

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']

        director_detail_list.append(person_info)
        
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

김대현 남자
1
김대현 남자
2
김대현 남자
3
김대현 남자
4
김대현 남자
5
김대현 남자
6
김대현 남자
7
김대현 남자
8
김대현 남자
9
김대현 남자
10
김대현 남자
11
김대현 남자
12
김대현 남자
13
김대현 남자
14
김대현 남자
15
김대현 남자
16
김대현 남자
17
김대현 남자
18
김대현 남자
19
김대현 남자
20
김대현 남자
21
김대현 남자
22
김대현 남자
23
김대현 남자
24
이상우 남자
25
이상우 남자
26
이상우 남자
27
이상우 남자
28
이상우 남자
29
이상우 남자
30
이상우 남자
31
이상우 남자
32
이상우 남자
33
이상우 남자
34
이상우 남자
35
이상우 남자
36
이상우 남자
37
이상우 남자
38
이상우 남자
39
이상우 남자
40
이상우 남자
41
이상우 남자
42
이상우 남자
43
이상우 남자
44
이상우 남자
45
이상우 남자
46
이상우 남자
47
이상우 남자
48
이상우 남자
49
이상우 남자
50
이상우 남자
51
이상우 남자
52
이상우 남자
53
이상우 남자
54
이상우 남자
55
이상우 남자
56
이상우 남자
57
이상우 남자
58
이상우 남자
59
이상우 남자
60
이상우 남자
61
이상우 남자
62
이상우 남자
63
이상우 남자
64
이상우 남자
65
이상우 남자
66
이상우 남자
67
이상우 남자
68
이상우 남자
69
이상우 남자
70
이상우 남자
71
이상우 남자
72
이상우 남자
73
이상우 남자
74
이상우 남자
75
이상우 남자
76
이상우 남자
77
이상우 남자
78
이상우 남자
79
이상우 남자
80
이상우 남자
81
이상우 남자
82
이상우 남자
83
이상우 남자
84
이상우 남자
85
이상우 남자
86
이상우 남자
87
이상우 남자
88
이상우 남자
89
이상우 남자
90
이상우 남자
91
이상우 남자
92
이상우 남자
93
이상우 남자
94
차성덕 여자
95
차성덕 여자
96
차성덕 여자
97
차성덕 여자
98
차성덕 여자
99
차성덕 여자
100
차성덕 여자
1

최국희 남자
769
최국희 남자
770
최국희 남자
771
최국희 남자
772
최국희 남자
773
최국희 남자
774
최국희 남자
775
최국희 남자
776
최국희 남자
777
최국희 남자
778
최국희 남자
779
장혜영 여자
780
장혜영 여자
781
장혜영 여자
782
장혜영 여자
783
장혜영 여자
784
장혜영 여자
785
장혜영 여자
786
정희재 여자
787
정희재 여자
788
정희재 여자
789
정희재 여자
790
정희재 여자
791
정희재 여자
792
정희재 여자
793
정희재 여자
794
정희재 여자
795
정희재 여자
796
정희재 여자
797
정희재 여자
798
정희재 여자
799
정희재 여자
800
정희재 여자
801
김서윤 여자
802
김서윤 여자
803
김서윤 여자
804
강이관 남자
805
강이관 남자
806
강이관 남자
807
강이관 남자
808
강이관 남자
809
강이관 남자
810
강이관 남자
811
강이관 남자
812
강이관 남자
813
강이관 남자
814
강이관 남자
815
강이관 남자
816
강이관 남자
817
부지영 여자
818
부지영 여자
819
부지영 여자
820
부지영 여자
821
부지영 여자
822
부지영 여자
823
부지영 여자
824
부지영 여자
825
부지영 여자
826
부지영 여자
827
부지영 여자
828
부지영 여자
829
부지영 여자
830
부지영 여자
831
부지영 여자
832
부지영 여자
833
부지영 여자
834
부지영 여자
835
부지영 여자
836
부지영 여자
837
부지영 여자
838
정대건 남자
839
정대건 남자
840
정대건 남자
841
정대건 남자
842
정대건 남자
843
정대건 남자
844
정대건 남자
845
정대건 남자
846
정대건 남자
847
정대건 남자
848
정대건 남자
849
정대건 남자
850
정대건 남자
851
정대건 남자
852
정대건 남자
853
정대건 남자
854
김중현 남자
855
김중현 남자
856
김중현 남자
857
김중현 남자
858
김중현 남자
859

이호경 남자
1471
송원근 남자
1472
유선동 남자
1473
유선동 남자
1474
유선동 남자
1475
유선동 남자
1476
유선동 남자
1477
유선동 남자
1478
유선동 남자
1479
유선동 남자
1480
유선동 남자
1481
한상호 남자
1482
한상호 남자
1483
한상호 남자
1484
한상호 남자
1485
한상호 남자
1486
한상호 남자
1487
한상호 남자
1488
김보라 여자
1489
김보라 여자
1490
김보라 여자
1491
김보라 여자
1492
김보라 여자
1493
김보라 여자
1494
김보라 여자
1495
김보라 여자
1496
김보라 여자
1497
김보라 여자
1498
이창근 남자
1499
이창근 남자
1500
박호찬 남자
1501
박호찬 남자
1502
박호찬 남자
1503
박호찬 남자
1504
박호찬 남자
1505
박호찬 남자
1506
박호찬 남자
1507
박호찬 남자
1508
박호찬 남자
1509
박호찬 남자
1510
박수진 None
1511
박수진 None
1512
시즈노 코분 남자
1513
시즈노 코분 남자
1514
시즈노 코분 남자
1515
시즈노 코분 남자
1516
시즈노 코분 남자
1517
시즈노 코분 남자
1518
시즈노 코분 남자
1519
시즈노 코분 남자
1520
조석현 남자
1521
조석현 남자
1522
조석현 남자
1523
조석현 남자
1524
조석현 남자
1525
조석현 남자
1526
조석현 남자
1527
강지은 남자
1528
강지은 남자
1529
강지은 남자
1530
강지은 남자
1531
강지은 남자
1532
강지은 남자
1533
김성기 None
1534
김성기 None
1535
고정욱 남자
1536
고정욱 남자
1537
고정욱 남자
1538
고정욱 남자
1539
고정욱 남자
1540
고정욱 남자
1541
고정욱 남자
1542
고정욱 남자
1543
강형철 남자
1544
강형철 남자
1545
강형철 남자
1546
강형철 남자
1547
강형철 남자
1548
강형철 남자
1549
강형철 남자
1550
강형철 남자
1

In [188]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

actor_detail_list = []
num = 1

for staffCd in actorCd_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        actor_detail_list.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

김정구 남자
김정구 남자
1
백성현 남자
백성현 남자
백성현 남자
백성현 남자
백성현 남자
백성현 남자
백성현 남자
백성현 남자
2
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
이상아 여자
3
서신애 여자
서신애 여자
서신애 여자
서신애 여자
서신애 여자
서신애 여자
서신애 여자
4
이정구 남자
이정구 남자
5
신원호 남자
신원호 남자
신원호 남자
신원호 남자
신원호 남자
6
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
서호철 남자
7
이다슬 여자
이다슬 여자
이다슬 여자
이다슬 여자
8
이산인 None
이산인 None
이산인 None
이산인 None
이산인 None
9
이은욱 남자
이은욱 남자
이은욱 남자
이은욱 남자
이은욱 남자
이은욱 남자
10
이찬양 남자
이찬양 남자
이찬양 남자
11
이철진 남자
이철진 남자
12
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
장기원 남자
13
장동주 남자
장동주 남자
장동주 남자
14
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
박정환 남자
15
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
김향기 여자
16
김호정 여자
김호정 여자
김호정 여자
김호정 여자

김충길 남자
김충길 남자
김충길 남자
김충길 남자
184
박원진 남자
박원진 남자
박원진 남자
185
백승환 남자
백승환 남자
백승환 남자
백승환 남자
백승환 남자
백승환 남자
백승환 남자
186
유현정 여자
유현정 여자
187
윤지혜 여자
윤지혜 여자
윤지혜 여자
윤지혜 여자
윤지혜 여자
188
이호정 None
이호정 None
이호정 None
이호정 None
이호정 None
이호정 None
이호정 None
이호정 None
이호정 None
189
조준희 None
조준희 None
조준희 None
조준희 None
조준희 None
190
조준희 None
조준희 None
조준희 None
조준희 None
조준희 None
191
고성완 남자
고성완 남자
고성완 남자
고성완 남자
고성완 남자
고성완 남자
192
우상전 남자
우상전 남자
우상전 남자
우상전 남자
우상전 남자
193
고동업 None
고동업 None
고동업 None
고동업 None
고동업 None
194
안홍진 남자
안홍진 남자
안홍진 남자
안홍진 남자
195
조용주 남자
조용주 남자
조용주 남자
조용주 남자
196
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
우지현 남자
197
이건우 남자
이건우 남자
198
정형석 남자
정형석 남자
정형석 남자
정형석 남자
정형석 남자
정형석 남자
정형석 남자
정형석 남자
정형석 남자
정형석 남자
199
문종원 남자
문종원 남자
문종원 남자
문종원 남자
문종원 남자
문종원 남자
문종원 남자
200
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달환 남자
조달

조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
조진웅 남자
288
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
주지훈 남자
289
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
김대현 남자
290
이경호 남자
이경호 남자
이경호 남자
291
이기용 남자
이기용 남자
292
이동희 남자
이동희 남자
이동희 남자
이동희 남자
이동희 남자
이동희 남자
이동희 남자
이동희 남자
이동희 남자
293
이상원 남자
이상원 남자
이상원 남자
이상원 남자
이상원 남자
이상원 남자
이상원 남자
이상원 남자
이상원 남자
294
이상현 남자
이상현 남자
이상현 남자
이상현 남자
이상현 남자
이상현 남자
이상현 남자
이상현 남자
이상현 남자
295
박현민 남자
박현민 남자
296
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
박혜영 여자
297
김재희 여자
김재희 여자
김재희 여자
298
이승찬 남자
이승찬 남자
이승찬 남자
299
이승환 남자
이승환 남자
이승환 남자


김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
김영민 남자
384
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
오하늬 여자
385
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
윤혜리 여자
386
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
설경구 남자
387
임시완 남자
임시완 남자
임시완 남자
임시완 남자
임시완 남자
임시완 남자
임시완 남자
임시완 남자
388
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
김희원 남자
389
전혜진 여자
전혜진 여자
전혜진 여자
전혜진 여자
전혜진 여자
전혜진 여자
전혜진

이원희 남자
이원희 남자
482
이은주 여자
이은주 여자
이은주 여자
이은주 여자
이은주 여자
이은주 여자
이은주 여자
483
손민석 남자
손민석 남자
손민석 남자
손민석 남자
손민석 남자
손민석 남자
484
손영진 남자
손영진 남자
손영진 남자
손영진 남자
손영진 남자
손영진 남자
485
이하윤 여자
이하윤 여자
486
김진구 남자
김진구 남자
김진구 남자
487
박경관 남자
박경관 남자
박경관 남자
박경관 남자
박경관 남자
488
박재민 남자
박재민 남자
489
박재영 남자
박재영 남자
박재영 남자
박재영 남자
박재영 남자
박재영 남자
박재영 남자
490
박정민 남자
박정민 남자
박정민 남자
박정민 남자
박정민 남자
491
권훈 남자
권훈 남자
권훈 남자
492
김기현 남자
김기현 남자
김기현 남자
김기현 남자
김기현 남자
김기현 남자
493
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
494
최준영 남자
최준영 남자
최준영 남자
최준영 남자
최준영 남자
최준영 남자
최준영 남자
최준영 남자
최준영 남자
최준영 남자
최준영 남자
495
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
조한철 남자
496
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
한창현 남자
497
엄효섭 남자
엄효섭 남자
엄

...

최지안 여자
최지안 여자
최지안 여자
최지안 여자
1238
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
김래원 남자
1239
원진아 여자
원진아 여자
원진아 여자
원진아 여자
원진아 여자
원진아 여자
원진아 여자
원진아 여자
1240
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
진선규 남자
1241
최귀화 남자
최귀화 남자
최귀화 남자
최귀화 남자
최귀화 남자
최귀화 남자
최귀화 남자
최귀화 남자
최귀화 남자
1242
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
최무성 남자
1243
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
주진모 남자
1244
장지건 남자
장지건 남자
장지건 남자
장지건 남자
장지건 남자
장지건 남자
장지건 남자
장지건 남자
장지건 남자
1245
정원중 남자
정

이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
이성욱 남자
1303
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
류경수 남자
1304
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
김고은 여자
1305
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
차래형 남자
1306
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
1307
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
박예영 여자
1308
나철 남자
나철 남자
나철 남자
나철 남자
나철 남자
나철 남자
나철 남자
나철 남자
나철 남자
나철 남자
나철 남자
1309
이해운 남자
이해운 남자
이해운 남자
이해운 남자
이해운 남자
이해운 남자
이해운 남자
이해운 남자
이해운 남자
이해운 남자
1310
박서준 남자
박서준 남자
박서준 남자
박서준 남자
박서준 남자
박서준 남자
박서준 남자
박서준 남자
1311
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자
안성기 남자


손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
손영순 여자
1375
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
라미란 여자
1376
이성경 여자
이성경 여자
이성경 여자
이성경 여자
이성경 여자
1377
윤상현 남자
윤상현 남자
윤상현 남자
윤상현 남자
1378
최수영 여자
최수영 여자
최수영 여자
최수영 여자
1379
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
염혜란 여자
1380
위하준 남자
위하준 남자
위하준 남자
위하준 남자
위하준 남자
위하준 남자
1381
주우재 남자
주우재 남자
1382
강홍석 남자
강홍석 남자
1383
김도완 남자
김도완 남자
김도완 남자
김도완 남자
1384
이정민 여자
이정민 여자
이정민 여자
이정민 여자
이정민 여자
이정민 여자
이정민 여자
이정민 여자
이정민 여자
1385
신현숙 여자
신현숙 여자
신현숙 여자
신현숙 여자


유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
유해진 남자
1432
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
윤계상 남자
1433
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
김홍파 남자
1434
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
우현 남자
1435
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 남자
김태훈 

양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
양현민 남자
1480
김기태 남자
김기태 남자
김기태 남자
김기태 남자
김기태 남자
김기태 남자
김기태 남자
김기태 남자
김기태 남자
1481
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
마동석 남자
1482
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
김상중 남자
1483
김아중 여자
김아중 여자
김아중 여자
김아중 여자
김아중 여자
김아중 여자
김아중 여자
김아중 여자
김아중 여자
김아중 여자
1484
장기용 남자
장기용 남자
장기용 남자
1485
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
강예원 여자
1486
김대영 남자
김대영 남자
김대영 남자
김대영 남자
김대영 남자
김대영 남자
1487
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
박형수 남자
1488
김혜윤 여자
김혜윤 여자
김혜윤

In [ ]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

staff_detail_list_1 = []
num = 1

for staffCd in staffCd1_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        staff_detail_list_1.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

In [ ]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

staff_detail_list_2 = []
num = 1

for staffCd in staffCd2_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        staff_detail_list_2.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

In [ ]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

staff_detail_list_3 = []
num = 1

for staffCd in staffCd3_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        staff_detail_list_3.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

<br>

### TODO. 영화인 상세정보를 뽑아주세요.

Q1) 2019 감독, 배우, 스태프 모두의 상세정보를 뽑아주세요.
각각의 리스트를 합칠 때는 아래 코드를 참고하세요.

```python
staff_detail_list_all = pd.DataFrame(staff_detail_list_1 + staff_detail_list_2 + staff_detail_list_3 + staff_detail_list_4 + staff_detail_list_5)
staff_detail_list_all
```

Q2) 가져온 데이터는 csv 형식, index=False로 저장해주세요.

In [184]:
director_detail_df = pd.DataFrame(director_detail_list)
director_detail_df.to_csv('director_detail_2019_ko.csv', index=False)

In [189]:
actor_detail_df = pd.DataFrame(actor_detail_list)
actor_detail_df.to_csv('actor_detail_2019_ko.csv', index=False)

In [ ]:
staff1_detail_df = pd.DataFrame(staff_detail_list_1)
staff1_detail_df.to_csv('staff1_detail_2019_ko.csv', index=False)

In [ ]:
staff2_detail_df = pd.DataFrame(staff_detail_list_2)
staff2_detail_df.to_csv('staff2_detail_2019_ko.csv', index=False)

In [ ]:
staff3_detail_df = pd.DataFrame(staff_detail_list_3)
staff3_detail_df.to_csv('staff3_detail_2019_ko.csv', index=False)

<br>\
<br>\
<br>

### 잘 돌아가다가 KeyError: 'peopleInfoResult'가 발생했을 때.

해당에러가 뜰 경우에는 key 사용 한도를 초과한 것입니다.

멈추기 전까지는 데이터가 다 저장되어있으니 당황하지 마시고 어디까지 저장되었는지 아래처럼 범위를 이용해서 또는 .tail() 함수를 이용해 데이터를 확인하고 다시 시작해야하는 부분이 어디인지 알아냅니다.

In [147]:
# 데이터 확인
staff_detail_list_2[-10:]

[{'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20040586',
  'movieNm': '돌려차기',
  'moviePartNm': '헤어팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20040586',
  'movieNm': '돌려차기',
  'moviePartNm': '특수분장'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20030107',
  'movieNm': '거울속으로',
  'moviePartNm': '분장팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20030107',
  'movieNm': '거울속으로',
  'moviePartNm': '헤어팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20020118',
  'movieNm': '재밌는 영화',
  'moviePartNm': '의상팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20010031',
  'movieNm': '수취인불명',
  'moviePartNm': '의상팀'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'movieCd': '20158554',
  'movieNm': '고백',
  'moviePartNm': '분장'},
 {'peopleCd': '10006383',
  'peopleNm': '김혜연',
  'sex': '여자',
  'm

In [110]:
# '김효주', '아메리카 타운' 정보가 맨 마지막일 경우
# index값을 확인한다. 3000까지 뽑힌거니까 3001부터 데이터를 다시 가져오면 됨.
peopleCd_list[(peopleCd_list['peopleNm'] == '김효주') & (peopleCd_list['movieNm'] == '아메리카 타운')]

,peopleCd,movieNm,peopleNm,repRoleNm,filmoNames
3000,20320553,아메리카 타운,김효주,분장팀,아메리카 타운
